In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Test accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10 # Numbers 0-9
dropout = 0.75 # probability to keep

Extracting .\train-images-idx3-ubyte.gz
Extracting .\train-labels-idx1-ubyte.gz
Extracting .\t10k-images-idx3-ubyte.gz
Extracting .\t10k-labels-idx1-ubyte.gz


In [7]:
# Layers and bias, let´s make space for them
weights = {
    'wc1': tf.Variable(tf.random_normal([5,5,1,32])),
    'wc2': tf.Variable(tf.random_normal([5,5,32,64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}
biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [8]:
# Redefine the conv2d in a way it applied all in the same step
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1,strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [9]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1,k,k,1], strides=[1,k,k,1], padding='SAME')

In [15]:
def conv_net(x, weights, biases, dropout):
    # Layer 1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)
    
    # Layer 2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)
    
    # Fully connected layer 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
    
    # Out layer
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out
    

In [17]:
# Let´s run it! create a session
x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
y = tf.placeholder(tf.float32, shape=[None, n_classes])

keep_prob = tf.placeholder(tf.float32)

logits = conv_net(x, weights, biases, dropout)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout
            })
            
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.
            })
            
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.
            })
            
            print('Epoch {:>2}, Batch{:>3}, -'
                  'Loss {:>10.4f} Validation accuracy: {:.6f}'.format(epoch +1, batch +1, loss, valid_acc))
            
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))
    

Epoch  1, Batch  1, -Loss 89295.7734 Validation accuracy: 0.085938
Epoch  1, Batch  2, -Loss 71676.9062 Validation accuracy: 0.074219
Epoch  1, Batch  3, -Loss 59904.9141 Validation accuracy: 0.097656
Epoch  1, Batch  4, -Loss 56000.8359 Validation accuracy: 0.085938
Epoch  1, Batch  5, -Loss 55171.0078 Validation accuracy: 0.097656
Epoch  1, Batch  6, -Loss 52956.9219 Validation accuracy: 0.085938
Epoch  1, Batch  7, -Loss 44873.0039 Validation accuracy: 0.113281
Epoch  1, Batch  8, -Loss 42758.8398 Validation accuracy: 0.082031
Epoch  1, Batch  9, -Loss 45716.0898 Validation accuracy: 0.097656
Epoch  1, Batch 10, -Loss 39582.0000 Validation accuracy: 0.121094
Epoch  1, Batch 11, -Loss 37169.3203 Validation accuracy: 0.125000
Epoch  1, Batch 12, -Loss 36223.8398 Validation accuracy: 0.117188
Epoch  1, Batch 13, -Loss 34738.3594 Validation accuracy: 0.117188
Epoch  1, Batch 14, -Loss 34711.9766 Validation accuracy: 0.121094
Epoch  1, Batch 15, -Loss 34900.4180 Validation accuracy: 0.13

Epoch  1, Batch124, -Loss 10730.4141 Validation accuracy: 0.386719
Epoch  1, Batch125, -Loss 11141.1504 Validation accuracy: 0.378906
Epoch  1, Batch126, -Loss  7225.3643 Validation accuracy: 0.351562
Epoch  1, Batch127, -Loss  9118.3984 Validation accuracy: 0.355469
Epoch  1, Batch128, -Loss  8431.3320 Validation accuracy: 0.410156
Epoch  1, Batch129, -Loss 10110.7119 Validation accuracy: 0.367188
Epoch  1, Batch130, -Loss  8179.1372 Validation accuracy: 0.417969
Epoch  1, Batch131, -Loss  8128.9644 Validation accuracy: 0.390625
Epoch  1, Batch132, -Loss 10089.4082 Validation accuracy: 0.382812
Epoch  1, Batch133, -Loss  7976.3887 Validation accuracy: 0.386719
Epoch  1, Batch134, -Loss  8951.7178 Validation accuracy: 0.390625
Epoch  1, Batch135, -Loss  9567.9775 Validation accuracy: 0.382812
Epoch  1, Batch136, -Loss 10145.0430 Validation accuracy: 0.371094
Epoch  1, Batch137, -Loss  8616.6807 Validation accuracy: 0.371094
Epoch  1, Batch138, -Loss 10328.2432 Validation accuracy: 0.41

Epoch  1, Batch247, -Loss  5260.8662 Validation accuracy: 0.496094
Epoch  1, Batch248, -Loss  5282.8115 Validation accuracy: 0.492188
Epoch  1, Batch249, -Loss  4938.2754 Validation accuracy: 0.500000
Epoch  1, Batch250, -Loss  5085.7280 Validation accuracy: 0.476562
Epoch  1, Batch251, -Loss  5134.8955 Validation accuracy: 0.480469
Epoch  1, Batch252, -Loss  4900.9697 Validation accuracy: 0.484375
Epoch  1, Batch253, -Loss  5155.0532 Validation accuracy: 0.484375
Epoch  1, Batch254, -Loss  5954.2627 Validation accuracy: 0.488281
Epoch  1, Batch255, -Loss  5154.1094 Validation accuracy: 0.519531
Epoch  1, Batch256, -Loss  5718.6714 Validation accuracy: 0.511719
Epoch  1, Batch257, -Loss  5858.1084 Validation accuracy: 0.484375
Epoch  1, Batch258, -Loss  5666.1924 Validation accuracy: 0.519531
Epoch  1, Batch259, -Loss  4467.4141 Validation accuracy: 0.558594
Epoch  1, Batch260, -Loss  5206.3438 Validation accuracy: 0.464844
Epoch  1, Batch261, -Loss  4349.0547 Validation accuracy: 0.50

Epoch  1, Batch370, -Loss  2667.7861 Validation accuracy: 0.574219
Epoch  1, Batch371, -Loss  3564.4868 Validation accuracy: 0.558594
Epoch  1, Batch372, -Loss  3677.5884 Validation accuracy: 0.496094
Epoch  1, Batch373, -Loss  2953.3972 Validation accuracy: 0.539062
Epoch  1, Batch374, -Loss  3075.5872 Validation accuracy: 0.531250
Epoch  1, Batch375, -Loss  3468.5444 Validation accuracy: 0.519531
Epoch  1, Batch376, -Loss  3796.9741 Validation accuracy: 0.550781
Epoch  1, Batch377, -Loss  4088.8057 Validation accuracy: 0.531250
Epoch  1, Batch378, -Loss  2833.3945 Validation accuracy: 0.542969
Epoch  1, Batch379, -Loss  4289.4966 Validation accuracy: 0.546875
Epoch  1, Batch380, -Loss  2991.2268 Validation accuracy: 0.523438
Epoch  1, Batch381, -Loss  4156.0146 Validation accuracy: 0.535156
Epoch  1, Batch382, -Loss  3738.0530 Validation accuracy: 0.507812
Epoch  1, Batch383, -Loss  3267.2368 Validation accuracy: 0.531250
Epoch  1, Batch384, -Loss  4045.6641 Validation accuracy: 0.54

Epoch  2, Batch 64, -Loss  2756.3142 Validation accuracy: 0.601562
Epoch  2, Batch 65, -Loss  3266.5088 Validation accuracy: 0.542969
Epoch  2, Batch 66, -Loss  3094.0994 Validation accuracy: 0.562500
Epoch  2, Batch 67, -Loss  2474.3328 Validation accuracy: 0.562500
Epoch  2, Batch 68, -Loss  3670.6545 Validation accuracy: 0.582031
Epoch  2, Batch 69, -Loss  3219.6118 Validation accuracy: 0.566406
Epoch  2, Batch 70, -Loss  2913.3997 Validation accuracy: 0.601562
Epoch  2, Batch 71, -Loss  3626.3994 Validation accuracy: 0.554688
Epoch  2, Batch 72, -Loss  2898.9434 Validation accuracy: 0.554688
Epoch  2, Batch 73, -Loss  2715.8794 Validation accuracy: 0.589844
Epoch  2, Batch 74, -Loss  2424.3157 Validation accuracy: 0.554688
Epoch  2, Batch 75, -Loss  3230.4111 Validation accuracy: 0.601562
Epoch  2, Batch 76, -Loss  2396.7498 Validation accuracy: 0.546875
Epoch  2, Batch 77, -Loss  3178.4197 Validation accuracy: 0.500000
Epoch  2, Batch 78, -Loss  2661.0542 Validation accuracy: 0.58

...

Epoch  9, Batch259, -Loss   584.8745 Validation accuracy: 0.660156
Epoch  9, Batch260, -Loss   701.6751 Validation accuracy: 0.683594
Epoch  9, Batch261, -Loss   504.1717 Validation accuracy: 0.671875
Epoch  9, Batch262, -Loss   724.7844 Validation accuracy: 0.710938
Epoch  9, Batch263, -Loss   510.3318 Validation accuracy: 0.632812
Epoch  9, Batch264, -Loss   698.0483 Validation accuracy: 0.675781
Epoch  9, Batch265, -Loss   528.5979 Validation accuracy: 0.679688
Epoch  9, Batch266, -Loss   865.3983 Validation accuracy: 0.710938
Epoch  9, Batch267, -Loss   547.0208 Validation accuracy: 0.695312
Epoch  9, Batch268, -Loss   638.5350 Validation accuracy: 0.664062
Epoch  9, Batch269, -Loss   718.6323 Validation accuracy: 0.679688
Epoch  9, Batch270, -Loss   708.1681 Validation accuracy: 0.660156
Epoch  9, Batch271, -Loss   713.9124 Validation accuracy: 0.636719
Epoch  9, Batch272, -Loss   664.9139 Validation accuracy: 0.671875
Epoch  9, Batch273, -Loss   749.7422 Validation accuracy: 0.69

Epoch  9, Batch382, -Loss   627.7448 Validation accuracy: 0.664062
Epoch  9, Batch383, -Loss   438.1578 Validation accuracy: 0.667969
Epoch  9, Batch384, -Loss   643.0732 Validation accuracy: 0.699219
Epoch  9, Batch385, -Loss   942.5312 Validation accuracy: 0.722656
Epoch  9, Batch386, -Loss   506.1628 Validation accuracy: 0.679688
Epoch  9, Batch387, -Loss   621.3791 Validation accuracy: 0.671875
Epoch  9, Batch388, -Loss   560.2499 Validation accuracy: 0.667969
Epoch  9, Batch389, -Loss   758.9847 Validation accuracy: 0.695312
Epoch  9, Batch390, -Loss   741.3582 Validation accuracy: 0.718750
Epoch  9, Batch391, -Loss   739.4843 Validation accuracy: 0.675781
Epoch  9, Batch392, -Loss   649.7292 Validation accuracy: 0.703125
Epoch  9, Batch393, -Loss   553.6091 Validation accuracy: 0.675781
Epoch  9, Batch394, -Loss   676.3103 Validation accuracy: 0.695312
Epoch  9, Batch395, -Loss   541.3962 Validation accuracy: 0.730469
Epoch  9, Batch396, -Loss   743.2111 Validation accuracy: 0.69

Epoch 10, Batch 76, -Loss   452.9831 Validation accuracy: 0.660156
Epoch 10, Batch 77, -Loss   594.7672 Validation accuracy: 0.699219
Epoch 10, Batch 78, -Loss   420.6645 Validation accuracy: 0.714844
Epoch 10, Batch 79, -Loss   626.3764 Validation accuracy: 0.695312
Epoch 10, Batch 80, -Loss   481.6269 Validation accuracy: 0.687500
Epoch 10, Batch 81, -Loss   559.9522 Validation accuracy: 0.683594
Epoch 10, Batch 82, -Loss   669.1203 Validation accuracy: 0.679688
Epoch 10, Batch 83, -Loss   635.4730 Validation accuracy: 0.699219
Epoch 10, Batch 84, -Loss   387.0293 Validation accuracy: 0.644531
Epoch 10, Batch 85, -Loss   525.9145 Validation accuracy: 0.687500
Epoch 10, Batch 86, -Loss   764.4559 Validation accuracy: 0.699219
Epoch 10, Batch 87, -Loss   798.8253 Validation accuracy: 0.695312
Epoch 10, Batch 88, -Loss   659.5970 Validation accuracy: 0.679688
Epoch 10, Batch 89, -Loss   587.7158 Validation accuracy: 0.648438
Epoch 10, Batch 90, -Loss   498.2849 Validation accuracy: 0.69

Epoch 10, Batch199, -Loss   425.2404 Validation accuracy: 0.683594
Epoch 10, Batch200, -Loss   476.4777 Validation accuracy: 0.726562
Epoch 10, Batch201, -Loss   487.6604 Validation accuracy: 0.667969
Epoch 10, Batch202, -Loss   408.7913 Validation accuracy: 0.660156
Epoch 10, Batch203, -Loss   502.1119 Validation accuracy: 0.675781
Epoch 10, Batch204, -Loss   517.9847 Validation accuracy: 0.683594
Epoch 10, Batch205, -Loss   425.1900 Validation accuracy: 0.691406
Epoch 10, Batch206, -Loss   843.9191 Validation accuracy: 0.667969
Epoch 10, Batch207, -Loss   739.6988 Validation accuracy: 0.675781
Epoch 10, Batch208, -Loss   586.2492 Validation accuracy: 0.687500
Epoch 10, Batch209, -Loss   500.5210 Validation accuracy: 0.664062
Epoch 10, Batch210, -Loss   421.6937 Validation accuracy: 0.726562
Epoch 10, Batch211, -Loss   677.6225 Validation accuracy: 0.691406
Epoch 10, Batch212, -Loss   498.1635 Validation accuracy: 0.695312
Epoch 10, Batch213, -Loss   749.5222 Validation accuracy: 0.71

Epoch 10, Batch322, -Loss   607.3752 Validation accuracy: 0.640625
Epoch 10, Batch323, -Loss   691.9496 Validation accuracy: 0.671875
Epoch 10, Batch324, -Loss   647.1464 Validation accuracy: 0.656250
Epoch 10, Batch325, -Loss   685.3390 Validation accuracy: 0.699219
Epoch 10, Batch326, -Loss   551.9196 Validation accuracy: 0.695312
Epoch 10, Batch327, -Loss   548.2728 Validation accuracy: 0.679688
Epoch 10, Batch328, -Loss   457.8510 Validation accuracy: 0.628906
Epoch 10, Batch329, -Loss   673.5718 Validation accuracy: 0.679688
Epoch 10, Batch330, -Loss   526.8395 Validation accuracy: 0.691406
Epoch 10, Batch331, -Loss   551.1177 Validation accuracy: 0.679688
Epoch 10, Batch332, -Loss   574.3793 Validation accuracy: 0.648438
Epoch 10, Batch333, -Loss   468.1445 Validation accuracy: 0.664062
Epoch 10, Batch334, -Loss   411.6219 Validation accuracy: 0.667969
Epoch 10, Batch335, -Loss   493.8644 Validation accuracy: 0.695312
Epoch 10, Batch336, -Loss   577.3886 Validation accuracy: 0.67